Cedric Keller***REMOVED*** & Alexander Zank***REMOVED***
# Assignment 5

This assignment is about visible light communication (VLC). LEDs act as transceivers and communicate using protocols that ensure no difference to "steady" light is visible to humans.

## Step 2

_What is the bandwidth of the optical spectrum?_

>The optical or visible spectrum is the range of wavelengths visible to the human eye.
>It ranges from 400THz (red) to around 800THz (violet). 
>In total, we therefore get a bandwidth of around 400THZ, which is a lot!  
>
>We can estimate the maximum throughput to be around 400 terabits/s using Stefan Mangold's rule of thumb.
  
  
_Is the visible spectrum regulated?_

>Concerning frequency, the visible spectrum is not regulated.
>Powerful light sources will, however, be regulated (such as strong lasers). 
  
  
_What is the difference between infrared and visible light?_

>Light in the frequency range of 300GHz to 400 THz is classified as infrared light.
>Infrared light has a lower frequency than visible light and is therefore not visible
>to humans. 
  
  
_Can infrared light penetrate water? Can visible light?_

>Looking at the graph: https://en.wikipedia.org/wiki/Electromagnetic_absorption_by_water#/media/File:Absorption_spectrum_of_liquid_water.png
>We can see that the visible spectrum is absorbed the most in water.
>This implies that visible light can barely penetrate water.
>50% of it is already absorbed within the first 10 meters. 
>Blue light is absorbed least and may reach depths of 100 meters.
>
>Infrared light is absorbed way less, which means it is able to penetrate much deeper.
  
  
_How do submarines communicate through the ocean when they operate below the water surface?_

>There are different possibilities depending on the depth of the submarine.
>If it is near the water surface, it may use (blue) light.
>
>Since light becomes less viable for lower depths,
>sonar (physical waves) might be used.
>
>There is also the possibility of extremely low-frequency radio waves,
>but these require giant antennas which consume enormous amounts of power.
  
  
_How can an LED be used as a receiver?_

>An LED behaves like a capacitor; It will slowly discharge after it is charged.
>The LED discharges faster if there is incoming light. This effect can be measured 
>and can therefore LEDs can detect incoming light.
  
  
_What are the benefits of using an LED instead of a photodiode as a receiver in consumer electronics?_

>In consumer electronics, cost plays a significant role. If we are able to reuse the LED as a receiver, 
>we do not have to add additional hardware in the form of a photodiode. 
>This massively reduces cost, especially in products where it is, for example, very expensive to add a hole
>for a photodiode. 
>With the LED as a receiver we simply have to change the microchip without further costs.
  
  



## Step 4

Using the provided ETH driver and microcontroller, we have implemented a two-way chat app powered by Python, "VLChat." Two devices connect to the microcontroller via serial and communicate by dispatching and reading streams of data transferred over light and processed by the microcontroller.

We use a threaded `Gateway` that dispatches and listens for messages over serial. The `CLI` runs on the main thread and communicated with the `Gateway` by means of two thread-safe queues. Unless the user enters input mode (by pressing the return key at any time), messages are printed to the console.

The setup is designed to be smart. For instance, serial ports are automatically discovered and sensible defaults provided. It also ensures that the system is correctly set-up by listening to affirmative responses.

Find the source code in the [GitHub Repository](https://github.com/AlexLike/ETH-WiNet-VLC-Lab).

| Overview  | Close-up  |
|---|---|
| ![](resources/vlchat-hw-overview.jpeg) | ![](resources/vlchat-hw-close.jpeg) |

| CLI (Setup)  | CLI (Messaging)  |
|---|---|
| ![](resources/vlchat-setup.png) | ![](resources/vlchat-interactive.png) |



## Step 5

Using our second application, "VLCollect," we measured several metrics' dependence on payload size and physical distance between the transceivers. Again, find the source code in the [GitHub Repository](https://github.com/AlexLike/ETH-WiNet-VLC-Lab).

